In [8]:
import sqlite3
import os
import re

# Open folder and directory
data = 'C:\\Users\\Wren\\Documents\\THEtool_logfiles'
filenames = []
dir_list = os.listdir(data)
for file in dir_list:
    if re.match(r"\d+-\w+-\S+", file):
        filenames.append(file)

# Empty a table in the log
for file in filenames:
    conn = sqlite3.connect(os.path.join(data, file))
    c = conn.cursor()
    c.execute("DELETE FROM props;")
    conn.commit()
    
# Clear tabs 
for file in filenames:
    conn = sqlite3.connect(os.path.join(data, file))
    c.execute("UPDATE act SET json='{\"is_focused\":false}' where k='focus' AND json LIKE '%is_focused\":false%'")
    conn.commit()

# Find the lowest value of t for each file
lowest_t = {}
for file in filenames:
    conn = sqlite3.connect(os.path.join(data, file))
    c = conn.cursor()
    c.execute("SELECT MIN(t) FROM (SELECT MIN(t) AS t FROM act UNION SELECT MIN(t) AS t FROM key UNION SELECT MIN(t) AS t FROM eye);")
    row = c.fetchone()
    if row is not None:
        lowest_t[file] = row[0]
    conn.close()

# Subtract the lowest t from all t values in each file
for file, lowt in lowest_t.items():
    conn = sqlite3.connect(os.path.join(data, file))
    c = conn.cursor()
    c.execute("UPDATE act SET t = t - ? WHERE t >= ?;", (lowt, lowt))
    c.execute("UPDATE key SET t = t - ? WHERE t >= ?;", (lowt, lowt))
    c.execute("UPDATE eye SET t = t - ? WHERE t >= ?;", (lowt, lowt))
    conn.commit()

# Change the name of the log
for file in filenames:   
    current_name = os.path.join(data, file)
    new_name = re.sub(r"(\d+-)\w+(-\S+)", r"\1anon\2", file)
    try:
        os.rename(current_name, os.path.join(data, new_name))
    except Exception as e:
        print(f"Skipping {current_name} - unable to rename file ({e})")
        continue

# Commit the changes and close the connection
conn.commit()
conn.close()

Skipping C:\Users\Wren\Documents\THEtool_logfiles\7000-iytxeq7u-xMB1FzNEjz - unable to rename file ([WinError 32] The process cannot access the file because it is being used by another process: 'C:\\Users\\Wren\\Documents\\THEtool_logfiles\\7000-iytxeq7u-xMB1FzNEjz' -> 'C:\\Users\\Wren\\Documents\\THEtool_logfiles\\7000-anon-xMB1FzNEjz')


In [11]:
# Open the connection and create a cursor
conn = sqlite3.connect(os.path.join(data, "7000-anon-xMB1FzNEjz"))
c = conn.cursor()

# Select the json column from the act table
c.execute("SELECT json FROM act")

# Fetch all rows and print the json column
rows = c.fetchall()
for row in rows:
    print(row[0])

# Close the connection
conn.close()

{"viewport_width":596,"viewport_height":944,"char_width":8.797,"char_height":22,"interline":2,"rows":39,"cols":67,"lpane_width":642,"rpane_width":642,"fb_width":300,"margin":10}
{"viewport_width":1327,"viewport_height":944,"char_width":19.243,"char_height":50,"interline":26,"rows":12,"cols":68,"lpane_width":252.99,"rpane_width":300,"fb_width":300,"margin":10}
{"npd":0}
{"npd":1}
{"npd":2}
{"npd":3}
{"npd":4}
{"npd":5}
{"npd":6}
{"npd":7}
{"npd":8}
{"npd":9}
{"npd":10}
{"npd":11}
{"npd":12}
{"npd":13}
{"npd":14}
{"npd":15}
{"npd":16}
{"npd":17}
{"npd":18}
{"cur_row":0,"cur_col":0,"top_row":0,"cur":{"npd":0,"offset":0}}
{"is_focused":true}
{"viewport_width":173,"viewport_height":301,"char_width":19.243,"char_height":50,"interline":26,"rows":3,"cols":8,"cur_row":0,"cur_col":0,"top_row":0,"lpane_width":0,"rpane_width":300,"fb_width":300,"margin":10}
{"viewport_width":173,"viewport_height":778,"char_width":19.243,"char_height":50,"interline":26,"rows":10,"cols":8,"cur_row":0,"cur_col":0,"to